# **1.Object Detection**

In [ ]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker

Python 3.10.12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 94.3 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
from google.colab import files
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/mediapipe/objdetection_dataset.zip

Archive:  /content/drive/MyDrive/mediapipe/objdetection_dataset.zip
   creating: objdetection_dataset/
   creating: objdetection_dataset/train/
   creating: objdetection_dataset/train/images/
  inflating: objdetection_dataset/train/images/00483aab-029_452a74d1.jpg  
  inflating: objdetection_dataset/train/images/0059d773-036_c58b80c0.jpg  
  inflating: objdetection_dataset/train/images/01ac8a16-IMG-20240119-WA0151.jpg  
  inflating: objdetection_dataset/train/images/01ff2988-006_209191e8.jpg  
  inflating: objdetection_dataset/train/images/0408434c-032_489447f3.jpg  
  inflating: objdetection_dataset/train/images/068889bf-IMG-20240119-WA0103.jpg  
  inflating: objdetection_dataset/train/images/075f6d88-024_b048bc9d.jpg  
  inflating: objdetection_dataset/train/images/082804d5-016_c5a64cbe.jpg  
  inflating: objdetection_dataset/train/images/0b131b4f-IMG-20240119-WA0109.jpg  
  inflating: objdetection_dataset/train/images/0baa3c4a-035_495d3f77.jpg  
  inflating: objdetection_dataset/tra

In [ ]:
train_dataset_path = "/content/objdetection_dataset/train"
validation_dataset_path = "/content/objdetection_dataset/validation"

In [ ]:
with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")

0: background
1: face
2: keyboard
3: laptop
4: microphone
5: monitor
6: mouse
7: xboxcontroller


In [ ]:
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)

train_data size:  286
validation_data size:  57


In [ ]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model',steps_per_epoch=100, learning_rate=0.6, epochs=100)
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams,
)

In [ ]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   144928    
                             , '4': (None, 16, 16, 1             
                             28),                 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


36/36 [==============================] - 69s 526ms/step - total_loss: 6.4463 - cls_loss: 6.0366 - box_loss: 0.0069 - model_loss: 6.3835 - val_total_loss: 1.5208 - val_cls_loss: 1.2420 - val_box_loss: 0.0043 - val_model_loss: 1.4579
Epoch 2/100
36/36 [==============================] - 7s 194ms/step - total_loss: 1.5543 - cls_loss: 1.2455 - box_loss: 0.0049 - model_loss: 1.4914 - val_total_loss: 1.4603 - val_cls_loss: 1.1776 - val_box_loss: 0.0044 - val_model_loss: 1.3975
Epoch 3/100
36/36 [==============================] - 6s 172ms/step - total_loss: 1.3864 - cls_loss: 1.1539 - box_loss: 0.0034 - model_loss: 1.3236 - val_total_loss: 1.3922 - val_cls_loss: 1.1106 - val_box_loss: 0.0044 - val_model_loss: 1.3294
Epoch 4/100
36/36 [==============================] - 7s 194ms/step - total_loss: 1.2426 - cls_loss: 1.0577 - box_loss: 0.0024 - model_loss: 1.1798 - val_total_loss: 1.2922 - val_cls_loss: 1.0267 - val_box_loss: 0.0041 - val_model_loss: 1.2294
Epoch 5/100
36/36 [====================

In [ ]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

15/15 [==============================] - 2s 37ms/step - total_loss: 0.4893 - cls_loss: 0.3455 - box_loss: 0.0016 - model_loss: 0.4272
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.16s).
Accumulating evaluation results...
DONE (t=0.07s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.665
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.903
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.702
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.666
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.713
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.763
 Average Recall     (AR) @[

In [ ]:
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

# **2.Object Detection Animal**

In [ ]:
!unzip /content/drive/MyDrive/mediapipe/animal_dataset.zip

Archive:  /content/drive/MyDrive/mediapipe/animal_dataset.zip
   creating: animal_dataset/
   creating: animal_dataset/train/
   creating: animal_dataset/train/images/
  inflating: animal_dataset/train/images/0115d6f0-016_266c2782.jpg  
  inflating: animal_dataset/train/images/011ce9de-040_45cea78f.jpg  
  inflating: animal_dataset/train/images/0175ce7c-044_d2628d90.jpg  
  inflating: animal_dataset/train/images/03187f4d-039_65e153dd.jpg  
  inflating: animal_dataset/train/images/0687f95b-024_87daea64.jpg  
  inflating: animal_dataset/train/images/06b090b9-017_ddfb507b.jpg  
  inflating: animal_dataset/train/images/07915076-007_2bfc5420.jpg  
  inflating: animal_dataset/train/images/07920b7b-024_78291eb1.jpg  
  inflating: animal_dataset/train/images/07a8c67e-015_e3f6ffd1.jpg  
  inflating: animal_dataset/train/images/09c694df-016_51ff9379.jpg  
  inflating: animal_dataset/train/images/0a2c0935-014_530f8321.jpg  
  inflating: animal_dataset/train/images/0b10480a-029_754ecd66.jpg  
  in

In [ ]:
train_dataset_path = "/content/animal_dataset/train"
validation_dataset_path = "/content/animal_dataset/validation"

In [ ]:
with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")

0: background
1: cat
2: dog
3: monkey
4: panda
5: penguin


In [ ]:
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)

train_data size:  240
validation_data size:  86


In [ ]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams,
)

In [ ]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   144928    
                             , '4': (None, 16, 16, 1             
                             28),                 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


30/30 [==============================] - 70s 534ms/step - total_loss: 5.0336 - cls_loss: 4.7019 - box_loss: 0.0054 - model_loss: 4.9707 - val_total_loss: 1.4105 - val_cls_loss: 1.1827 - val_box_loss: 0.0033 - val_model_loss: 1.3476
Epoch 2/30
30/30 [==============================] - 6s 203ms/step - total_loss: 1.3974 - cls_loss: 1.1657 - box_loss: 0.0034 - model_loss: 1.3345 - val_total_loss: 1.3073 - val_cls_loss: 1.0742 - val_box_loss: 0.0034 - val_model_loss: 1.2444
Epoch 3/30
30/30 [==============================] - 5s 167ms/step - total_loss: 1.2097 - cls_loss: 1.0190 - box_loss: 0.0026 - model_loss: 1.1467 - val_total_loss: 1.1485 - val_cls_loss: 0.9037 - val_box_loss: 0.0036 - val_model_loss: 1.0855
Epoch 4/30
30/30 [==============================] - 6s 198ms/step - total_loss: 0.9646 - cls_loss: 0.8043 - box_loss: 0.0019 - model_loss: 0.9016 - val_total_loss: 0.9634 - val_cls_loss: 0.7425 - val_box_loss: 0.0032 - val_model_loss: 0.9003
Epoch 5/30
30/30 [========================

In [ ]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

22/22 [==============================] - 3s 80ms/step - total_loss: 0.7028 - cls_loss: 0.4971 - box_loss: 0.0029 - model_loss: 0.6398
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.24s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.438
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.667
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.519
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.443
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.611
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.674
 Average Recall     (AR) @[ 

In [ ]:
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


float_ckpt.data-00000-of-00001	float_ckpt.index  metadata.json  model.tflite


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>